#Análise dos Dados de Conexão dos Usuários


##Pendências

* Sem pendências

##Análise do Levantamento Realizado


Fonte de dados: **Sistema Radius**

Variáveis levantadas preliminarmente:


*   Tempo médio de conexão
*   Consumo mensal de dados
*   Fabricante do equipamenteo
*   Total de conexões no mês
*   Percentual de disponibilidade da conexão
*   Tipo de acesso (streaming, jogos, web)

**Análise das variáveis**<p>
Após a preparação e análise dos dados, optou-se por considerar apenas as variáveis: 
*   **Consumo mensal de dados**
*   **Total de conexões por mês**
*   **Percentual de disponibilidade**
*   **Fabricante do equipamento**

A variável *Tempo médio de conexão* não foi utilizada por que ela está diretamente relacionada com as variáveis *Total de conexões por mês* e *Percentual de disponibilidade*, então optou-se por adotar essas duas.

A variável *Fabricante do equipamento* não foi utilizada por que não existe uma relação única entre o usuário e um único fabricante de equipamento, então optou-se por não adotar essa informação para o cálculo da probabilidade de churn.

**Oportunidades Identificadas**<p>
Estudar a relação entre o equipamento utilizado e o número de conexões mensais e o tempo médio de conexão.

Estudar a relação entre a causa da desconexão e outras variáveis, tais como: equipamento utilizado, bairro, cidade, meio de transmissão (fibra ou rádio).

Estudar a relação entre o número de conexões e o bairro, cidade e meio de transmissão (fibra ou radio).


##Importação de bibliotecas

In [ ]:
import pandas as pd
import re as re

##Definição de Funções e Constantes

In [ ]:
#constantes
HORAS = 'HORAS'
MINUTOS = 'MINUTOS'

In [ ]:
def calcula_tempo_conexao(data_inicio, data_fim, unidade = 'HORAS'):

  if (pd.notnull(data_fim) == False):
    data_fim = pd.Timestamp.today(tz=data_inicio.tz)

  dias = pd.Timedelta(data_fim - data_inicio).days
  horas = pd.Timedelta(data_fim - data_inicio).seconds / (60 * 60)    # valor em horas
  minutos = pd.Timedelta(data_fim - data_inicio).seconds / 60         # minutos

  if unidade == HORAS:
    resultado = round((dias * 24) + horas) 
  else:
    resultado = round((dias * 24 * 60) + minutos)
  
  return resultado

In [ ]:
def calcula_perc_disponibilidade(dados_conexoes, username):

  #filtra os dados das conexões do usuário
  dados_conexao_usuario = dados_conexoes.loc[[username], :]

  #guarda a data da primeira conexão do usuário
  data_inicio = dados_conexao_usuario['data_inicio'].min()
  
  #guarda a data atual
  data_fim = pd.Timestamp.today(tz=data_inicio.tz)

  #calcula o tempo decorrido desde a primeira conexão
  total_horas = calcula_tempo_conexao(data_inicio, data_fim)
  
  #calcula o tempo de conexão do usuário
  horas_conexao_usuario = dados_conexao_usuario['tempo_conexao'].sum()

  perc_disponibilidade = round((horas_conexao_usuario / total_horas), 4) 

  if perc_disponibilidade > 1:
      perc_disponibilidade = 1

  return perc_disponibilidade

In [ ]:
def retorna_fabricante_endereco_mac(dados_mac_address, mac_address, utilizar_somente_prefixo = True):

  try:
    #retira caracteres não alfanuméricos e corta o mac no 6o caracter
    if utilizar_somente_prefixo:
      token_src = re.sub('[\W_]+', '', mac_address).upper()[:6]  
    else:
      token_src = mac_address

    #realiza a busca pelos primeiros 6 caracteres
    dados_aux = dados_mac_address.loc[token_src]
    
    nro_registros = dados_aux.size

    if nro_registros == 1:
      return dados_aux.values[0]
    else:
      return None

  except KeyError:
    return None
  except TypeError:
    return None
  else:
    return None

In [ ]:
def retorna_fabricante_usuario(dados_conexoes, username):
  #filtra os dados das conexões do usuário
  dados_conexao_usuario = dados_conexoes.loc[[username], :]

  #pega o primeiro registro das conexões do usuário
  mac_equipamento = dados_conexao_usuario.iloc[0]['mac_equipamento']

  #consulta o fabricante do endereço mac do equipamento do usuário
  fabricante = retorna_fabricante_endereco_mac(dados_mac_address, mac_equipamento)

  return fabricante

##Carga dos Dados

In [ ]:
#dataset com dados das conexões
dados_origem = pd.read_csv('conexoes.csv')
dados_origem.head()

,username,nasportid,acctstarttime,acctstoptime,acctinputoctets,acctoutputoctets,callingstationid,acctterminatecause
0,roberta.lins,ae1:300,2021-09-27 21:59:29.000 -0300,NaN,358,623,00-e0-4c-e7-0d-7f,NaN
1,patricia.aguiar,ae1.demux0.3221225482:503,2021-09-27 21:59:17.000 -0300,NaN,124,244,d8-77-8b-d9-c4-b4,NaN
2,margarida.cruz,ae1.demux0.3221233669:314,2021-09-27 21:58:59.000 -0300,NaN,829,638,d8-77-8b-04-0d-b9,NaN
3,marines.7712.2,ae1.demux0.3221249199:810,2021-09-27 21:57:49.000 -0300,2021-09-27 21:57:50.000 -0300,0,0,58-10-8c-36-b6-22,User-Request
4,fernando.jesus,ae1.demux0.3221340515:301,2021-09-27 21:56:43.000 -0300,NaN,0,30,68-ff-7b-8c-e3-d9,NaN


In [ ]:
linhas, colunas = dados_origem.shape
print('A fonte de dados de conexões possui %d linhas e %d colunas.' % (linhas, colunas))

A fonte de dados de conexões possui 100000 linhas e 8 colunas.


In [ ]:
#dataset contendo a relação de prefixos de endereço mac e seu respectivo fabricante
dados_mac_address_original = pd.read_csv('conexoes.macaddress.fabricante.csv')
dados_origem.head()

,username,nasportid,acctstarttime,acctstoptime,acctinputoctets,acctoutputoctets,callingstationid,acctterminatecause
0,roberta.lins,ae1:300,2021-09-27 21:59:29.000 -0300,NaN,358,623,00-e0-4c-e7-0d-7f,NaN
1,patricia.aguiar,ae1.demux0.3221225482:503,2021-09-27 21:59:17.000 -0300,NaN,124,244,d8-77-8b-d9-c4-b4,NaN
2,margarida.cruz,ae1.demux0.3221233669:314,2021-09-27 21:58:59.000 -0300,NaN,829,638,d8-77-8b-04-0d-b9,NaN
3,marines.7712.2,ae1.demux0.3221249199:810,2021-09-27 21:57:49.000 -0300,2021-09-27 21:57:50.000 -0300,0,0,58-10-8c-36-b6-22,User-Request
4,fernando.jesus,ae1.demux0.3221340515:301,2021-09-27 21:56:43.000 -0300,NaN,0,30,68-ff-7b-8c-e3-d9,NaN


In [ ]:
linhas, colunas = dados_mac_address_original.shape
print('A fonte de dados de endereços mac e fabricantes possui %d linhas e %d colunas.' % (linhas, colunas))

A fonte de dados de endereços mac e fabricantes possui 43889 linhas e 8 colunas.


In [ ]:
#dataset com informações dos usuários que possuem o flashman instalado
dados_usuario_flashman = pd.read_csv('conexoes.flashman.csv')
dados_usuario_flashman.head()

,username
0,abel.nascimento
1,abiron.luz
2,acenimartins
3,acksom.bertotto
4,adair.arouca


In [ ]:
linhas, colunas = dados_usuario_flashman.shape
print('A fonte de dados de usuários com flashmann instalado possui %d linhas e %d colunas.' % (linhas, colunas))

A fonte de dados de usuários com flashmann instalado possui 1304 linhas e 1 colunas.


##Tratamento dos Dados

###Dados das Conexões

In [ ]:
#dataset principal da análise com as informações de conexões do usuário
dados = dados_origem.copy()

In [ ]:
#Criação de novas colunas
dados['data_inicio'] = pd.to_datetime(dados['acctstarttime'], errors='coerce')
dados['data_fim'] = pd.to_datetime(dados['acctstoptime'], errors='coerce')
dados['ano_mes'] = dados['data_inicio'].dt.strftime("%Y-%m")
dados['mac_equipamento'] = dados['callingstationid']
dados['consumo_dados'] = dados['acctinputoctets'] + dados['acctoutputoctets']
dados['tempo_conexao'] = dados.apply(lambda x: calcula_tempo_conexao(x['data_inicio'], x['data_fim']), axis=1) 

dados.head()

,username,nasportid,acctstarttime,acctstoptime,acctinputoctets,acctoutputoctets,callingstationid,acctterminatecause,data_inicio,data_fim,ano_mes,mac_equipamento,consumo_dados,tempo_conexao
0,roberta.lins,ae1:300,2021-09-27 21:59:29.000 -0300,NaN,358,623,00-e0-4c-e7-0d-7f,NaN,2021-09-27 21:59:29-03:00,NaT,2021-09,00-e0-4c-e7-0d-7f,981,713
1,patricia.aguiar,ae1.demux0.3221225482:503,2021-09-27 21:59:17.000 -0300,NaN,124,244,d8-77-8b-d9-c4-b4,NaN,2021-09-27 21:59:17-03:00,NaT,2021-09,d8-77-8b-d9-c4-b4,368,713
2,margarida.cruz,ae1.demux0.3221233669:314,2021-09-27 21:58:59.000 -0300,NaN,829,638,d8-77-8b-04-0d-b9,NaN,2021-09-27 21:58:59-03:00,NaT,2021-09,d8-77-8b-04-0d-b9,1467,713
3,marines.7712.2,ae1.demux0.3221249199:810,2021-09-27 21:57:49.000 -0300,2021-09-27 21:57:50.000 -0300,0,0,58-10-8c-36-b6-22,User-Request,2021-09-27 21:57:49-03:00,2021-09-27 21:57:50-03:00,2021-09,58-10-8c-36-b6-22,0,0
4,fernando.jesus,ae1.demux0.3221340515:301,2021-09-27 21:56:43.000 -0300,NaN,0,30,68-ff-7b-8c-e3-d9,NaN,2021-09-27 21:56:43-03:00,NaT,2021-09,68-ff-7b-8c-e3-d9,30,713


In [ ]:
#exclui colunas não utilizadas
dados.drop(['acctstarttime', 'acctstoptime', 'nasportid', 'callingstationid'], axis=1, inplace=True)

#renomeia o nome das colunas
dados.columns = ['username', 'qtde_upload', 'qtde_download', 'causa_encerramento', 'data_inicio', 'data_fim', 'ano_mes', 'mac_equipamento', 'consumo_dados', 'tempo_conexao']

#reorganiza as colunas 
dados = dados[['username', 'ano_mes', 'data_inicio', 'data_fim', 'tempo_conexao', 'causa_encerramento', 'qtde_download', 'qtde_upload', 'consumo_dados', 'mac_equipamento']]

dados.head()

,username,ano_mes,data_inicio,data_fim,tempo_conexao,causa_encerramento,qtde_download,qtde_upload,consumo_dados,mac_equipamento
0,roberta.lins,2021-09,2021-09-27 21:59:29-03:00,NaT,713,NaN,623,358,981,00-e0-4c-e7-0d-7f
1,patricia.aguiar,2021-09,2021-09-27 21:59:17-03:00,NaT,713,NaN,244,124,368,d8-77-8b-d9-c4-b4
2,margarida.cruz,2021-09,2021-09-27 21:58:59-03:00,NaT,713,NaN,638,829,1467,d8-77-8b-04-0d-b9
3,marines.7712.2,2021-09,2021-09-27 21:57:49-03:00,2021-09-27 21:57:50-03:00,0,User-Request,0,0,0,58-10-8c-36-b6-22
4,fernando.jesus,2021-09,2021-09-27 21:56:43-03:00,NaT,713,NaN,30,0,30,68-ff-7b-8c-e3-d9


In [ ]:
#dataset para consultar os dados das conexões do usuário
dados_busca = dados.set_index('username')

###Dados dos Endereços MACs e Respectivos Fabricantes

In [ ]:
#cria a coluna sem os caracteres especiais do endereço mac
dados_mac_address = dados_mac_address_original
dados_mac_address['prefixo_mac'] = dados_mac_address['oui'].str.replace('[^a-zA-Z0-9]', '').str.slice(0,6)

#exclui as colunas não necessárias e cria o índice pelo prefixo do endereço mac
dados_mac_address = dados_mac_address[['prefixo_mac', 'companyName']]
dados_mac_address.set_index('prefixo_mac', inplace=True)

###Dados do Usuário

In [ ]:
#prepara o dataset que irá agrupar o resultado final por usuário
dados_usuario = pd.DataFrame(dados['username'].unique())
dados_usuario.columns = ['username']
dados_usuario.set_index('username', inplace=True)

print(f'Número de {dados_usuario.shape[0]} usuários no total.')

Número de 7554 usuários no total.


###Dados Usuários com Flashman instalados

In [ ]:
#cria a coluna para sinalizar usuário com flashman instalados
dados_usuario_flashman['flashman_instalado'] = 1

dados_usuario_flashman.set_index('username', inplace=True)

##Cálculo das Variáveis

###Número de conexões e consumo mensais

In [ ]:
#agrupa as informações das conexões por usuário e ano/mês
dados_usuario_mes_download = dados.groupby(['username', 'ano_mes'])['qtde_download'].agg(['count', 'sum'])
dados_usuario_mes_upload = dados.groupby(['username', 'ano_mes'])['qtde_upload'].agg(['sum'])

dados_usuario_mes_download.columns = ['conexoes', 'qtde_download']
dados_usuario_mes_upload.columns = ['qtde_upload']

dados_usuario_mes = dados_usuario_mes_download.join(dados_usuario_mes_upload)

dados_usuario_mes.reset_index(inplace=True)

dados_usuario_mes.head()

,username,ano_mes,conexoes,qtde_download,qtde_upload
0,.michalczuk,2021-09,9,61673985360,1367473488
1,2k.eireli,2021-09,5,171110661085,79863443632
2,Jessicacs,2021-09,2,6685360012,180447620
3,Kleine,2021-09,2,89533792783,17463280778
4,NEIDEFH,2021-09,17,22392804231,1127920668


###Percentual de disponibilidade

In [ ]:
#percorre o dataset de usuário e cria um dicionario com o valor dos respectivos percentuais de disponibilidade
dicionario = {}
for item in dados_usuario.index:
  dicionario.update({item: calcula_perc_disponibilidade(dados_busca, item)})

#cria um dataframe a partir do dicionário para permitir o join na próxima linha
dados_temp = pd.DataFrame.from_dict(dicionario, orient='index', columns=['perc_disponibilidade'])
dados_temp.index.name = 'username'

dados_usuario_disponibilidade = dados_usuario.join(dados_temp)

dados_usuario_disponibilidade.head()

,perc_disponibilidade
username,
roberta.lins,1.000
patricia.aguiar,1.000
margarida.cruz,1.000
marines.7712.2,0.000
fernando.jesus,0.987


**Verificar se o cálculo levará em conta as horas ou minutos para considerar indisponibilidade.**

###Fabricante do equipamento do usuário

In [ ]:
#análise para verificar os usuários com mais do que um equipamento
dados_grupo_usuario_mac_equipamento = dados.groupby(['username', 'mac_equipamento']).agg('count').reset_index()[['username', 'mac_equipamento']]

dados_qtde_equipamento_usuario = pd.DataFrame(dados_grupo_usuario_mac_equipamento['username'].value_counts()).reset_index()
dados_qtde_equipamento_usuario.columns = ['username', 'qtde_equip']

dados_grupo_usuario_mac_equipamento.set_index('username', inplace=True)

#dataset com a lista de usuários com apenas 1 equipamento
dados_usuario_mac_equipamento = dados_qtde_equipamento_usuario.query('qtde_equip == 1')
dados_usuario_mac_equipamento.set_index('username', inplace=True)

print('Total de %s linhas agrupando usuário.'%dados_grupo_usuario_mac_equipamento.shape[0])
print('Total de %s usuários com mais do que um equipamento.'%dados_qtde_equipamento_usuario.query('qtde_equip > 1').shape[0])
print('Total de %s usuários com apenas um equipamento.'%dados_usuario_mac_equipamento.shape[0])

Total de 7748 linhas agrupando usuário.
Total de 110 usuários com mais do que um equipamento.
Total de 7444 usuários com apenas um equipamento.


**Observação:**<p>
Como alguns poucos usuários possuem mais do que um equipamento, foi necessário um tratamento para desconsiderar essa informação desses usuários.






In [ ]:
#percorre o dataset de usuário e cria um dicionario com o valor dos respectivos endereços mac
dicionario = {}
for item in dados_usuario.index:
  #dicionario.update({item: retorna_fabricante_usuario(dados_busca, item)})
  dicionario.update({item: retorna_fabricante_usuario(dados_grupo_usuario_mac_equipamento, item)})
  
dados_temp = pd.DataFrame.from_dict(dicionario, orient='index', columns=['fabricante_equipamento'])
dados_temp.index.name = 'username'

dados_usuario_fabricante_equipamento = dados_usuario.join(dados_temp)
dados_usuario_fabricante_equipamento['fabricante_equipamento'].fillna('Vários', inplace=True)

###Usuários com Flashman instalado

In [ ]:
#análise para verificar os usuários com mais do que um equipamento
dados_usuario_flashman = dados_usuario.join(dados_usuario_flashman)
dados_usuario_flashman.fillna(0, inplace=True)

In [ ]:
dados_usuario_flashman.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7554 entries, roberta.lins to dalila.pamplona
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   flashman_instalado  7554 non-null   float64
dtypes: float64(1)
memory usage: 438.0+ KB


##Preparação do Dataset Final

In [ ]:
#Agrupa por usuário para calcular a média mensal das conexões e consumo de dados
dados_usuário_conexao_consumo = dados_usuario_mes.groupby(['username'])[['conexoes', 'qtde_download', 'qtde_upload']].agg('mean').round(3)
dados_usuário_conexao_consumo.columns = ['conexoes_mes', 'download_mes', 'upload_mes']

dados_usuario = dados_usuario.join(dados_usuário_conexao_consumo)

#Mescla com o dataset do percentual de disponibilidade dos serviços
dados_usuario = dados_usuario.join(dados_usuario_disponibilidade) 

#Mescla com o dataset do fabricante do equipamento do usuário
dados_usuario = dados_usuario.join(dados_usuario_fabricante_equipamento)

#Mescla com o dataset com informações dos usuários com flashman instalado
dados_usuario = dados_usuario.join(dados_usuario_flashman)

dados_usuario.head()

,conexoes_mes,download_mes,upload_mes,perc_disponibilidade,fabricante_equipamento,flashman_instalado
username,,,,,,
roberta.lins,19,364941750494,19550345860,1.000,Realtek Semiconductor Corp,0.0
patricia.aguiar,4,198568382462,22309479746,1.000,intelbras,0.0
margarida.cruz,10,184472137348,5432349228,1.000,intelbras,0.0
marines.7712.2,21,0,0,0.000,intelbras,0.0
fernando.jesus,5,26191880033,3541586258,0.987,"TP-Link Tech Co, Ltd",0.0


In [ ]:
dados_usuario.to_csv('dados_usuario.csv', sep=';', decimal=',')

##Códigos utilizados durante o estudo

In [ ]:
#Colunas usadas durante o estudo
#dados['mac_equipamento'] = dados['callingstationid'].str.replace('[^a-zA-Z0-9]', '')
#dados['fabricante_equipamento'] = dados.apply(lambda x: retorna_fabricante_equipamento(dados_mac_address, x['mac_equipamento']), axis=1)
#dados['tempo_conexao'] = dados.apply(lambda x: calcula_tempo_conexao(x['data_inicio'], x['data_fim']), axis=1) 

#Verificação de conexões ativas
#dados_conexoes_ativas = dados[dados["data_fim"].notnull()]
#dados_conexoes_ativas.count()

In [ ]:
#Dataset para consulta do fabricante pela endereço mac

# dados_mac_address = pd.read_csv('macaddress.fabricante.csv')
# dados_mac_address['prefixo_mac'] = dados_mac_address['oui'].str.replace('[^a-zA-Z0-9]', '').str.slice(0,6)
# dados_mac_address = dados_mac_address[['prefixo_mac', 'companyName']]
# dados_mac_address.set_index('prefixo_mac', inplace=True)

# #prepara o dataset que vai vincular o mac address ao fabricante
# lista_mac_address = dados_origem['callingstationid'].unique()

# dados_fabric_mac_address = pd.DataFrame({'endereco_mac': lista_mac_address})

# dados_fabric_mac_address['fabricante'] = dados_fabric_mac_address.apply(
#     lambda x: retorna_fabricante_equipamento(dados_mac_address, x['endereco_mac']), axis=1)

# dados_fabric_mac_address.set_index('endereco_mac', inplace=True)

In [ ]:
#Associa o fabricante ao equipamento da conexão
# dados_teste = pd.DataFrame()

# dados_teste['fabricante'] = dados.apply(
#     lambda x: retorna_fabricante_equipamento(dados_fabric_mac_address, x['mac_equipamento'], False), axis=1)

In [ ]:
# #prepara o dataset que vai vincular o mac address das conexões aos seus fabricantes
# lista_mac_address_conexao = dados_origem['callingstationid'].unique()
# dados_mac_address_conexao = pd.DataFrame({'endereco_mac': lista_mac_address_conexao})

# dados_mac_address_conexao['fabricante'] = dados_mac_address_conexao.apply(
#     lambda x: retorna_fabricante_endereco_mac(dados_mac_address, x['endereco_mac']), axis=1)

# dados_mac_address_conexao.head()